In [281]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
from itertools import chain

In [282]:
def split_list_idx(olist): #old list, first list, second list
    idx = []
    for i in range(len(olist)):
        length = len(olist[i])
        if(length == 0):
            idx.append(i)
            
    for i in range(len(idx)):
        if(i != (len(idx)-1)):
            if(idx[i]+1 == idx[i+1]):
                continue
            else:
                first_row = idx[i]+1
                last_row = idx[i+1]
                idx[i] = first_row
                
    del idx[0:len(idx)-4]
    return (idx)
    

In [283]:
def get_hw_sw_data(url): #, row1, row2
    html = urlopen(url)
    soup = BeautifulSoup(html, "lxml")
    
    column_headers = ['Platform', 'Year', 'Change', 'Total']

    data_rows = soup.findAll('tr')#[row1:row2] # skip the first 2 header rows  #406 413

    company_data = [[td.getText() for td in data_rows[i].findAll('td')] for i in range(len(data_rows))]

    new_company_data = []  # create an empty list to hold all the data

    for i in range(len(data_rows)):  # for each table row
        company_row = []  # create an empty list for each pick/company

        # for each table data element from each table row
        for td in data_rows[i].findAll('td'):        
            # get the text content and append to the company_row         
            tdtext = []
            for i in td.getText():
                if i=="\n" or i=="\t":
                    continue
                else:
                    tdtext.append(i)
           
            tdtext = "".join(tdtext)    
            company_row.append(tdtext)        
        
        
        if(len(company_row) == 0 or len(company_row) == 4):
            new_company_data.append(company_row)
        else:
            continue    
    
    idx = split_list_idx(new_company_data)
    hardware = new_company_data[idx[len(idx)-4]:idx[len(idx)-3]]
    software = new_company_data[idx[len(idx)-2]:idx[len(idx)-1]]
    
    return(column_headers, hardware, software)

In [284]:
def output(url):
    data = get_hw_sw_data(url)
    colheader = data[0]
    hw = data[1]
    sw = data[2]
    df = pd.DataFrame(sw, columns=colheader)
    return(df)

In [285]:
#output excel
writer = pd.ExcelWriter('Global Software Sales by Platform.xlsx')
for i in range(13, 0, -1):
    year = i + 4
    if(year < 10):
        yr = '0'+str(year)
    else:
        yr = str(year)
        
    url = 'http://www.vgchartz.com/yearly/20'+yr+'/Global/'
    df = output(url)
    df.to_excel(writer,'20'+yr)
writer.save()       